## Accessing DM-EFD data


In this notebook we demonstrate how to extract data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import aioinflux
import pandas as pd
from jupyterlabutils.notebook import EfdClient

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

To access the EFD, you will need to put a file called `.lsst/notebook_auth.yaml` in your home directory.  It should be formatted in the following way (substituting the appropriate values, of course).  Ping Anglo or Simon on Slack (`@afausti`, `@ksk`) if you have any problem.

```yaml
test:
  username: "user"
  host: "endpoint.edu"
  password: "passwd"
```

In [ ]:
client = EfdClient('lab_efd')

## Listing topics
Topics are mapped to [InfluxDB measurements](https://docs.influxdata.com/influxdb/v1.7/concepts/key_concepts/#measurement). The following query simply lists all topics in the InfluxDB `efd` database:

In [ ]:
topics = await client.get_topics()
topics

## Querying topic data
[InfluxQL](https://docs.influxdata.com/influxdb/v1.7/query_language/spec/), the influxDB query language, is very similar to SQL.

In [ ]:
t2 = pd.Timestamp.now(tz='UTC')
t1 = t2 - pd.Timedelta('1d')

In [ ]:
df = await client.select_time_series('lsst.sal.ATCamera.wreb', '*', t1, t2)
df.head()

At this point you can use [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) to analyze the data. First plot all CCD temperature data at full resolution.

In [ ]:
df.plot(y='ccdTemp0')

Now average data over 15 minute intervals and plot.

In [ ]:
ccdTemp01h = df['ccdTemp0'].resample('15min').mean()

In [ ]:
ccdTemp01h.plot()

Finally, we can look at correlations between values in the database.

In [ ]:
df.plot.scatter(x='temp1', y='temp2')

## Querying annotations

In Chronograf, you can annotate the time-series to mark interesting events or features in the data. These annotations are saved in the `chronograf` database and can also be queried.

In [ ]:
df = await client.client.query('SELECT * FROM "chronograf"."autogen"."annotations"')
df['text']